**Install dependencies**

In [2]:
!pip install pylda2vec
!pip install numpy==1.16.1
!pip install jellyfish

You should consider upgrading via the 'c:\users\test\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\test\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\test\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


**Mount google drive**

In [2]:
#https://stackoverflow.com/questions/47744131/colaboratory-can-i-access-to-my-google-drive-folder-and-file
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

**Import packages**

In [3]:
import logging
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import spacy
from lda2vec import preprocess, Corpus
logging.basicConfig()

ModuleNotFoundError: No module named 'chainer'

In [3]:
np.version.version

'1.16.1'

**Data preprocessing**

In [4]:
# Fetch data
remove = ('headers', 'footers', 'quotes')
texts = fetch_20newsgroups(subset='train', remove=remove).data
# Remove tokens with these substrings
bad = set(["ax>", '`@("', '---', '===', '^^^'])


def clean(line):
    line = line.lower()
    to_remove2 = "[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]:"
    table2 = str.maketrans("", "", to_remove2)
    line = line.translate(table2)
    line = line.replace('\r','')
    line = line.replace('\n',' ')
    line = line.replace('\p','')
    line = ''.join([i for i in line if not i.isdigit()])
    return ' '.join(w for w in line.split() if not any(t in w for t in bad))

In [5]:
max_length = 10000   # Limit of 10k words per document
# Convert to unicode (spaCy only works with unicode)
texts = [str(clean(d)) for d in texts if len(str(clean(d))) > 0]

In [6]:
# tokens, vocab = preprocess.tokenize(texts, max_length, merge=False,n_threads=4)


In [6]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

**Generating tokens,vocabulary and corpus**

In [7]:
tokens = np.load("/content/drive/AI meeting assistant/tokens.npy")
vocab = np.load("/content/drive/AI meeting assistant/vocab.npy")
 

In [8]:
vocab = vocab.tolist()

In [10]:
# np.save('/content/drive/AI meeting assistant/tokens.npy', tokens)
# np.save('/content/drive/AI meeting assistant/vocab.npy', vocab)

In [9]:
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=30)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)

In [19]:
pickle.dump(vocab, open('/content/drive/AI meeting assistant/vocab.pkl', 'wb'))
pickle.dump(corpus, open('/content/drive/AI meeting assistant/corpus.pkl', 'wb'))
np.save("/content/drive/AI meeting assistant/flattened", flattened)
np.save("/content/drive/AI meeting assistant/doc_ids", doc_ids)
np.save("/content/drive/AI meeting assistant/pruned", pruned)
np.save("/content/drive/AI meeting assistant/bow", bow)

In [10]:
assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 300


In [11]:
fn_wordvc = '/content/drive/AI meeting assistant/GoogleNews-vectors-negative300.bin'

In [12]:
vectors, s, f = corpus.compact_word_vectors(vocab, filename=fn_wordvc)
np.save("/content/drive/AI meeting assistant/vectors", vectors)

**lda2vec model training**

In [13]:
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec

In [14]:
fn_vocab = '/content/drive/AI meeting assistant/vocab.pkl'
fn_corpus = '/content/drive/AI meeting assistant/corpus.pkl'
fn_flatnd = '/content/drive/AI meeting assistant/flattened.npy'
fn_docids = '/content/drive/AI meeting assistant/doc_ids.npy'
fn_vectors = '/content/drive/AI meeting assistant/vectors.npy'
vocab = pickle.load(open(fn_vocab, 'rb'))
corpus = pickle.load(open(fn_corpus, 'rb'))
flattened = np.load(fn_flatnd)
doc_ids = np.load(fn_docids)
vectors = np.load(fn_vectors)

In [15]:
# Number of documents
n_docs = doc_ids.max() + 1
# Number of unique words in the vocabulary
n_vocab = flattened.max() + 1
# 'Strength' of the dircihlet prior; 200.0 seems to work well
clambda = 200.0
# Number of topics to fit
n_topics = int(os.getenv('n_topics', 20))
batchsize = 4096
# Power for neg sampling
power = float(os.getenv('power', 0.75))
# Intialize with pretrained word vectors
pretrained = bool(int(os.getenv('pretrained', True)))
# Sampling temperature
temperature = float(os.getenv('temperature', 1.0))
# Number of dimensions in a single word vector
n_units = int(os.getenv('n_units', 300))
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_vocab]
# How many tokens are in each document
doc_idx, lengths = np.unique(doc_ids, return_counts=True)
doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
doc_lengths[doc_idx] = lengths
# Count all token frequencies
tok_idx, freq = np.unique(flattened, return_counts=True)
term_frequency = np.zeros(n_vocab, dtype='int32')
term_frequency[tok_idx] = freq

In [16]:
for key in sorted(locals().keys()):
    val = locals()[key]
    if len(str(val)) < 100 and '<' not in str(val):
        print(key, val)

Out {3: '1.16.1'}
_ 1.16.1
_3 1.16.1
__ 
___ 
__doc__ Automatically created module for IPython interactive environment
__loader__ None
__name__ __main__
__package__ None
__spec__ None
_dh ['/content']
_exit_code 0
_i10 assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 300
_i11 fn_wordvc = '/content/drive/AI meeting assistant/GoogleNews-vectors-negative300.bin'
_i3 np.version.version
_i8 vocab = vocab.tolist()
_oh {3: '1.16.1'}
bad {'---', '===', 'ax>', '^^^', '`@("'}
batchsize 4096
clambda 200.0
doc_ids [    0     0     0 ... 11003 11003 11003]
doc_idx [    0     1     2 ... 11001 11002 11003]
doc_lengths [ 86  80 288 ... 103  52  40]
f 1772
flattened [   8   28 1197 ...  149   45 3786]
fn_corpus /content/drive/AI meeting assistant/corpus.pkl
fn_docids /content/drive/AI meeting assistant/doc_ids.npy
fn_flatnd /content/drive/AI meeting assistant/flattened.npy
fn_vectors /content/drive/AI meeting assistant/vectors.npy
fn_vocab /content/drive/AI meeting assistant/v

In [17]:
model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                n_samples=15, power=power, temperature=temperature)

In [18]:
if os.path.exists('/content/drive/AI meeting assistant/lda2vec.hdf5'):
    print("Reloading from saved")
    serializers.load_hdf5("/content/drive/AI meeting assistant/lda2vec.hdf5", model)
    
if pretrained:
    model.sampler.W.data[:, :] = vectors[:n_vocab, :]

In [19]:
optimizer = O.Adam()
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)

In [20]:
j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')

In [21]:
for epoch in range(1):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    if j % 100 == 0 and j > 100:
        coherence = topic_coherence(top_words)
        for j in range(n_topics):
            print(j, coherence[(j, 'cv')])
        kw = dict(top_words=top_words, coherence=coherence, epoch=epoch)
        progress[str(epoch)] = pickle.dumps(kw)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    np.savez('topics.pyldavis', **data)
    print(epoch)
    for d, f in utils.chunks(batchsize, doc_ids, flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy())
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1
    serializers.save_hdf5("/content/drive/AI meeting assistant/lda2vec.hdf5", model)

Top words in topic 0 joe mental debate hockey clinton trip pope appearance attack shooting
Top words in topic 1 obey handguns illegal accepting soviet burning purposes inform sexual rpm
Top words in topic 2 dpi blah jpeg pleased et mit believer gr skipjack glad
Top words in topic 3 ships tyre pirates genocide adaptec resolution businesses villages antigun ports
Top words in topic 4 physician clinic doctor miracle mountain mercury medical baptism impossible automatically
Top words in topic 5 kg dpi delta carter speculation gm championships cylinder playoff cc
Top words in topic 6 dc chipset chi denning thank frequencies vesa cryptography k latin
Top words in topic 7 gd password dept resistor article accelerator answered raid tyre mailer
Top words in topic 8 slide jersey million mountain percent convince subscribe billion citizenship championships
Top words in topic 9 recchi jagr vesa henrik dept mice puck brake gene braking
Top words in topic 10 lamb slaughter gene buffalo apostles bull

In [22]:
npz = np.load(open('topics.pyldavis.npz', 'rb'))
dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()
top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

Topic 0 joe mental debate hockey clinton trip pope appearance attack shooting
Topic 1 obey handguns illegal accepting soviet burning purposes inform sexual rpm
Topic 2 dpi blah jpeg pleased et mit believer gr skipjack glad
Topic 3 ships tyre pirates genocide adaptec resolution businesses villages antigun ports
Topic 4 physician clinic doctor miracle mountain mercury medical baptism impossible automatically
Topic 5 kg dpi delta carter speculation gm championships cylinder playoff cc
Topic 6 dc chipset chi denning thank frequencies vesa cryptography k latin
Topic 7 gd password dept resistor article accelerator answered raid tyre mailer
Topic 8 slide jersey million mountain percent convince subscribe billion citizenship championships
Topic 9 recchi jagr vesa henrik dept mice puck brake gene braking
Topic 10 lamb slaughter gene buffalo apostles bull exploration meter wolverine popup
Topic 11 torque rpm motion rev schism asynchronous stephanopoulos throttle joystick orthodox
Topic 12 dir su